In [169]:
import os.path
import shelve
from collections import OrderedDict

import numpy as np
import scipy.optimize
from prettytable import PrettyTable

In [214]:
shelf = 'route_opt.db'
data = {}
try:
    with shelve.open(shelf) as db:
        data[1] = db['morning']
        data[2] = db['afternoon']
except:
    data[1] = {}
    data[2] = {}

In [215]:
route = {}
route[1] = OrderedDict({'Hill St': [25, 40, 30],
                        'Hegeman Bridge Rd': [30, 50, 40],
                        'Rt. 40': [45, 60, 50],
                        'Sarles Ferry': [40, 50, 45],
                        'CR-113': [45, 60, 50],
                        'Stillwater Bridge Rd': [25, 40, 35],
                        'Stillwater': [25, 40, 30],
                        'Hill in Stillwater': [25, 40, 35],
                        'Rt. 4': [35, 45, 40],
                        'Mechanicville': [25, 40, 30],
                        'Leaving Mechanicville': [25, 45, 35],
                        '45 mph Rt. 4': [40, 55, 45],
                        'Rt. 146 hill': [40, 60, 45],
                        'Rt. 146 curve': [45, 60, 50],
                        'Rt. 146 before light': [45, 60, 45],
                        'Rt. 146 after light': [45, 55, 45],
                        'Rt. 146 Halfmoon': [45, 55, 45],
                        'Clifton Park': [40, 50, 45],
                        'Hill by Vent Fitness': [40, 60, 45],
                        '55 mph Rt. 146': [45, 60, 50],
                        'Rt. 146 near Balltown': [40, 60, 45],
                        'Balltown in Rexford': [40, 50, 45],
                        'Balltown on bridge': [40, 50, 45],
                        '45 mph Balltown': [40, 50, 45],
                        '40 mph Balltown': [40, 50, 40],
                        'River Rd': [35, 50, 40],
                       })

In [216]:
route[2] = OrderedDict({'River Rd': [35, 50, 40],
                        '40 mph Balltown': [40, 50, 40],
                        '45 mph Balltown': [40, 50, 45],
                        'Balltown on bridge': [40, 50, 45],
                        'Balltown in Rexford': [40, 50, 45],
                        'Rt. 146 near Balltown': [40, 60, 45],
                        '55 mph Rt. 146': [45, 60, 50],
                        'Hill by Vent Fitness': [40, 60, 45],
                        'Clifton Park': [40, 50, 45],
                        'Rt. 146 Halfmoon': [45, 55, 45],
                        'Rt. 146 before light': [45, 60, 45],
                        'Rt. 146 after light': [45, 60, 45],
                        'Rt. 146 curve': [45, 60, 50],
                        'Rt. 146 hill': [40, 60, 45],
                        '45 mph Rt. 4': [40, 55, 45],
                        'Entering Mechanicville': [30, 45, 35],
                        'Mechanicville': [25, 40, 30],
                        'Rt. 4': [35, 45, 40],
                        'Hill in Stillwater': [25, 40, 35],
                        'Stillwater': [25, 40, 30],
                        'Stillwater Bridge Rd': [25, 40, 35],
                        'CR-113': [45, 60, 50],
                        'Sarles Ferry': [40, 50, 45],
                        'Rt. 40': [45, 60, 50],
                        'Hegeman Bridge Rd': [30, 50, 40],
                        'Hill St': [25, 40, 30],
                       })

In [217]:
x0 = {}
x0[1] = [bounds[2] for bounds in route[1].values()]
x0[2] = [bounds[2] for bounds in route[2].values()]

In [224]:
ITER = 0
def f(x, dset):
    global ITER
    dat = data[dset]
    rte = route[dset]
    x = tuple(int(v) for v in np.rint(x))
    if ITER in dat:
        y = dat[ITER]
        print(f'{ITER} MPG for {x} = {y}')
    else:
        tbl = PrettyTable(['Segment', 'Target Speed'])
        tbl.align['Segment'] = 'l'
        for seg, val in zip(rte, x):
            tbl.add_row([seg, val])
        print(tbl)
        print('ITER =', ITER)
        raise StopIteration
    ITER += 1
    return 10000 / y

In [225]:
def add(dset, x, y):
    data[dset][x] = y
    with shelve.open(shelf) as db:
        name = 'morning' if dset == 1 else 'afternoon'
        db[name] = data[dset]

In [226]:
def opt(dset):
    global ITER
    ITER = 0
    try:
        return scipy.optimize.minimize(f, x0[dset], args=(dset,), method='SLSQP',
                                       options={'ftol': 1, 'disp': True, 'eps': 1},
                                       bounds=list(x[:2] for x in route[dset].values()))
    except:
        pass

In [227]:
def clear():
    with shelve.open(shelf) as db:
        db.clear()

In [232]:
def display():
    with shelve.open(shelf) as db:
        for key, val in db.items():
            print(f'{key}:', val)

In [228]:
opt(1)

+-----------------------+--------------+
| Segment               | Target Speed |
+-----------------------+--------------+
| Hill St               |      30      |
| Hegeman Bridge Rd     |      40      |
| Rt. 40                |      50      |
| Sarles Ferry          |      45      |
| CR-113                |      50      |
| Stillwater Bridge Rd  |      35      |
| Stillwater            |      30      |
| Hill in Stillwater    |      35      |
| Rt. 4                 |      40      |
| Mechanicville         |      30      |
| Leaving Mechanicville |      35      |
| 45 mph Rt. 4          |      45      |
| Rt. 146 hill          |      45      |
| Rt. 146 curve         |      50      |
| Rt. 146 before light  |      45      |
| Rt. 146 after light   |      45      |
| Rt. 146 Halfmoon      |      45      |
| Clifton Park          |      45      |
| Hill by Vent Fitness  |      45      |
| 55 mph Rt. 146        |      50      |
| Rt. 146 near Balltown |      45      |
| Balltown in Re

In [229]:
opt(2)

+------------------------+--------------+
| Segment                | Target Speed |
+------------------------+--------------+
| River Rd               |      40      |
| 40 mph Balltown        |      40      |
| 45 mph Balltown        |      45      |
| Balltown on bridge     |      45      |
| Balltown in Rexford    |      45      |
| Rt. 146 near Balltown  |      45      |
| 55 mph Rt. 146         |      50      |
| Hill by Vent Fitness   |      45      |
| Clifton Park           |      45      |
| Rt. 146 Halfmoon       |      45      |
| Rt. 146 before light   |      45      |
| Rt. 146 after light    |      45      |
| Rt. 146 curve          |      50      |
| Rt. 146 hill           |      45      |
| 45 mph Rt. 4           |      45      |
| Entering Mechanicville |      35      |
| Mechanicville          |      30      |
| Rt. 4                  |      40      |
| Hill in Stillwater     |      35      |
| Stillwater             |      30      |
| Stillwater Bridge Rd   |      35

In [230]:
add(1, 0, 34.2)

In [231]:
add(2, 0, 34.3)

In [234]:
clear()